In [30]:
from SymbolicDSGE import ModelConfig, ModelParser, DSGESolver
import sympy as sp
from warnings import catch_warnings, simplefilter
import numpy as np

In [31]:
conf = ModelParser("MODELS/POST82.yaml").get()

In [32]:
with catch_warnings():
    simplefilter(action="ignore")
    mat = sp.Matrix(conf.equations.model)
mat

Matrix([
[                             Eq(Pi(t), beta*Pi(t + 1) + kappa*x(t) + z(t))],
[                  Eq(x(t), -tau_inv*(-Pi(t + 1) + r(t)) + g(t) + x(t + 1))],
[Eq(r(t), e_R + rho_r*r(t - 1*1) + (1 - rho_r)*(psi_pi*Pi(t) + psi_x*x(t)))],
[                                          Eq(g(t), e_g + rho_g*g(t - 1*1))],
[                                          Eq(z(t), e_z + rho_z*z(t - 1*1))]])

In [33]:
sol = DSGESolver(conf)
comp = sol.compile(variable_order=conf.variables, n_state=3, n_exog=2)
conf.variables

[g, z, r, Pi, x]

In [34]:
solved = sol.solve(
    comp,
    steady_state=np.asarray([0.0, 0.0, 0.0, 0.0, 0.0], dtype=float),
    log_linear=False,
)

C:\Users\guney\Documents\GitHub\ThesisDraft\SymbolicDSGE\solver.py:245: ComplexWarning: Casting complex values to real discards the imaginary part
  # x_{t+1} = hx x_t + eta eps_{t+1}
C:\Users\guney\Documents\GitHub\ThesisDraft\SymbolicDSGE\solver.py:246: ComplexWarning: Casting complex values to real discards the imaginary part
  # y_t = gx x_t


In [35]:
solved.policy.eig

array([0.27920118+0.j, 0.83000003+0.j, 0.84999992+0.j, 2.56517116+0.j,
       1.18470582+0.j])

In [36]:
params = {
    p.name: float(conf.calibration.parameters[p])
    for p in conf.parameters
    if p in conf.calibration.parameters
}

# state at time t
s = np.array([0.05, 0.077, 0.06])  # or any test state
P = solved.policy.p
F = solved.policy.f

# controls at time t (jump variables)
c = F @ s

cur = np.concatenate([s, c])

# expected next state (NO shock)
s1 = P @ s
c1 = F @ s1
fwd = np.concatenate([s1, c1])

res = solved.compiled.equations(fwd, cur, params)
print(res)

[np.complex128(-1.0494781432779376e-09+0j)
 np.complex128(8.938198320374013e-09+0j) -e_R - 4.89569941617557e-9
 1.52736900210781e-9 - e_g -e_z - 6.02379433833899e-9]


In [37]:
solved.irf(["g", "z"], 10)

{'g': array([0.        , 1.        , 0.83000003, 0.68890005, 0.57178706,
        0.47458328, 0.39390414, 0.32694045, 0.27136058, 0.22522929,
        0.18694032]),
 'z': array([0.        , 1.        , 0.84999992, 0.72249987, 0.61412483,
        0.52200606, 0.44370511, 0.37714931, 0.32057688, 0.27249032,
        0.23161675]),
 'r': array([0.        , 0.        , 0.48060113, 0.53692545, 0.48745108,
        0.41903176, 0.35418962, 0.29776799, 0.24991054, 0.20964626,
        0.17585806]),
 'Pi': array([0.        , 4.43311278, 1.36437731, 0.48877605, 0.2282891 ,
        0.14191864, 0.10618574, 0.08631514, 0.07234203, 0.06126655,
        0.05206564]),
 'x': array([ 0.        ,  3.59030921,  0.05256747, -0.79263794, -0.90747443,
        -0.8365712 , -0.72926093, -0.62491858, -0.53256677, -0.45304956,
        -0.38517891]),
 '_X': array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 1.        ,  1.        ,  0.        ,  4.43311278,  3.59030921],
        [ 0.83000